In [2]:
# 필요한 패키지 불러오기
import pandas as pd
from konlpy.tag import Okt 
okt = Okt()
import gensim
import gensim.corpora as corpora
import os
# os.environ["PYTHONIOENCODING"] = "utf-8”
import pyLDAvis.gensim

In [3]:
# 파일 불러오기
df1 = pd.read_excel('Apple1.xlsx')
df2 = pd.read_excel('Apple2.xlsx')
df3 = pd.read_excel('Apple3.xlsx')
df4 = pd.read_excel('Apple4.xlsx')
df5 = pd.read_excel('Apple5.xlsx')

# 열 방향으로 데이터프레임 병합
combined_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

# total data 저장
combined_df.to_excel('ToTApple.xlsx', index=False)


In [4]:
# 저장한 데이터 불러오기
df = pd.read_excel('ToTApple.xlsx')
df.head()

,상품명,구매자 이름,구매자 평점,리뷰 제목,리뷰 내용,맛 만족도
0,"Apple 에어팟, MMEF2KH/A, 혼합 색상",팔반점수니,5,비싸서 고민되면 사는게 맞다.,1. 배송속도 / 2. 제조년월 / 3. 포장 품질 / 4. 새제품? / 5. 구매...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,"
1,"Apple 에어팟, MMEF2KH/A, 혼합 색상",최*,4,배송은 엉망이지만 사용 후기는 아주 만족,와.. 대박 방금 받았는데...엄청 안전하게 배송해 주셨네요. 후기는 차차 더 추가...,만족도 평가 없음
2,"Apple 에어팟, MMEF2KH/A, 혼합 색상",솔직한마음,5,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다.,아이폰6s에서 8로 바꿨더니 이어폰잭이 없어서 엄청 불편하더군요...그래서 와이어 ...,만족도 평가 없음
3,"Apple 에어팟, MMEF2KH/A, 혼합 색상",이*빈,4,진심 이건 신세계다!!!!!!!!@_@,그동안 쿠팡에서 로켓배송을 시켜도 한번도 박스손상없었는데 ㅠㅠ 좀 찌그러져서 온게 ...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,"
4,"Apple 에어팟, MMEF2KH/A, 혼합 색상",다이욱트,5,애플의 감성을 담은 블루투스 이어폰,무선 블루투스 이어폰의 새강자 '에어팟'그렇다. 애플이 만들면 우리가 흔히 알던 것...,만족도 평가 없음


In [5]:
# 리뷰 제목과 내용 모두 리뷰이기에 합쳐준다.
df['review'] = df['리뷰 제목'] + df['리뷰 내용']
df

,상품명,구매자 이름,구매자 평점,리뷰 제목,리뷰 내용,맛 만족도,review
0,"Apple 에어팟, MMEF2KH/A, 혼합 색상",팔반점수니,5,비싸서 고민되면 사는게 맞다.,1. 배송속도 / 2. 제조년월 / 3. 포장 품질 / 4. 새제품? / 5. 구매...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,",비싸서 고민되면 사는게 맞다.1. 배송속도 / 2. 제조년월 / 3. 포장 품질 /...
1,"Apple 에어팟, MMEF2KH/A, 혼합 색상",최*,4,배송은 엉망이지만 사용 후기는 아주 만족,와.. 대박 방금 받았는데...엄청 안전하게 배송해 주셨네요. 후기는 차차 더 추가...,만족도 평가 없음,배송은 엉망이지만 사용 후기는 아주 만족와.. 대박 방금 받았는데...엄청 안전하게...
2,"Apple 에어팟, MMEF2KH/A, 혼합 색상",솔직한마음,5,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다.,아이폰6s에서 8로 바꿨더니 이어폰잭이 없어서 엄청 불편하더군요...그래서 와이어 ...,만족도 평가 없음,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다.아이폰6s에서 8로 바꿨더니 ...
3,"Apple 에어팟, MMEF2KH/A, 혼합 색상",이*빈,4,진심 이건 신세계다!!!!!!!!@_@,그동안 쿠팡에서 로켓배송을 시켜도 한번도 박스손상없었는데 ㅠㅠ 좀 찌그러져서 온게 ...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,",진심 이건 신세계다!!!!!!!!@_@그동안 쿠팡에서 로켓배송을 시켜도 한번도 박스...
4,"Apple 에어팟, MMEF2KH/A, 혼합 색상",다이욱트,5,애플의 감성을 담은 블루투스 이어폰,무선 블루투스 이어폰의 새강자 '에어팟'그렇다. 애플이 만들면 우리가 흔히 알던 것...,만족도 평가 없음,애플의 감성을 담은 블루투스 이어폰무선 블루투스 이어폰의 새강자 '에어팟'그렇다. ...
...,...,...,...,...,...,...,...
7490,Apple 2021 에어팟 3세대 블루투스 이어폰,박*지,5,좋습니디,등록된 리뷰내용이 없습니다,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 성...",좋습니디등록된 리뷰내용이 없습니다
7491,Apple 2021 에어팟 3세대 블루투스 이어폰,이*민,5,역시 애플,에어팟은 애플케어 드세요 금방 고장나요,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 적...",역시 애플에어팟은 애플케어 드세요 금방 고장나요
7492,Apple 2021 에어팟 3세대 블루투스 이어폰,임*현,5,말해뭐해 에어팟인데 ~,등록된 리뷰내용이 없습니다,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,가성비: 성...",말해뭐해 에어팟인데 ~등록된 리뷰내용이 없습니다
7493,Apple 2021 에어팟 3세대 블루투스 이어폰,예쁜파란,5,좋아요..성능..최고입니다,등록된 리뷰내용이 없습니다,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,사용시간: ...",좋아요..성능..최고입니다등록된 리뷰내용이 없습니다


In [6]:
df = df.rename(columns = {'구매자 평점':'rating'})
df.head()

,상품명,구매자 이름,rating,리뷰 제목,리뷰 내용,맛 만족도,review
0,"Apple 에어팟, MMEF2KH/A, 혼합 색상",팔반점수니,5,비싸서 고민되면 사는게 맞다.,1. 배송속도 / 2. 제조년월 / 3. 포장 품질 / 4. 새제품? / 5. 구매...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,",비싸서 고민되면 사는게 맞다.1. 배송속도 / 2. 제조년월 / 3. 포장 품질 /...
1,"Apple 에어팟, MMEF2KH/A, 혼합 색상",최*,4,배송은 엉망이지만 사용 후기는 아주 만족,와.. 대박 방금 받았는데...엄청 안전하게 배송해 주셨네요. 후기는 차차 더 추가...,만족도 평가 없음,배송은 엉망이지만 사용 후기는 아주 만족와.. 대박 방금 받았는데...엄청 안전하게...
2,"Apple 에어팟, MMEF2KH/A, 혼합 색상",솔직한마음,5,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다.,아이폰6s에서 8로 바꿨더니 이어폰잭이 없어서 엄청 불편하더군요...그래서 와이어 ...,만족도 평가 없음,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다.아이폰6s에서 8로 바꿨더니 ...
3,"Apple 에어팟, MMEF2KH/A, 혼합 색상",이*빈,4,진심 이건 신세계다!!!!!!!!@_@,그동안 쿠팡에서 로켓배송을 시켜도 한번도 박스손상없었는데 ㅠㅠ 좀 찌그러져서 온게 ...,"디자인: 아주 마음에 들어요,착용감: 아주 좋아요,음질: 아주 만족해요,",진심 이건 신세계다!!!!!!!!@_@그동안 쿠팡에서 로켓배송을 시켜도 한번도 박스...
4,"Apple 에어팟, MMEF2KH/A, 혼합 색상",다이욱트,5,애플의 감성을 담은 블루투스 이어폰,무선 블루투스 이어폰의 새강자 '에어팟'그렇다. 애플이 만들면 우리가 흔히 알던 것...,만족도 평가 없음,애플의 감성을 담은 블루투스 이어폰무선 블루투스 이어폰의 새강자 '에어팟'그렇다. ...


In [7]:
df2 = df[['rating', 'review']]
df2

,rating,review
0,5,비싸서 고민되면 사는게 맞다.1. 배송속도 / 2. 제조년월 / 3. 포장 품질 /...
1,4,배송은 엉망이지만 사용 후기는 아주 만족와.. 대박 방금 받았는데...엄청 안전하게...
2,5,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다.아이폰6s에서 8로 바꿨더니 ...
3,4,진심 이건 신세계다!!!!!!!!@_@그동안 쿠팡에서 로켓배송을 시켜도 한번도 박스...
4,5,애플의 감성을 담은 블루투스 이어폰무선 블루투스 이어폰의 새강자 '에어팟'그렇다. ...
...,...,...
7490,5,좋습니디등록된 리뷰내용이 없습니다
7491,5,역시 애플에어팟은 애플케어 드세요 금방 고장나요
7492,5,말해뭐해 에어팟인데 ~등록된 리뷰내용이 없습니다
7493,5,좋아요..성능..최고입니다등록된 리뷰내용이 없습니다


In [8]:
# 리뷰 데이터만 가져오기
review = df['review']

# 빈 리스트를 만들어 명사 토큰을 추출해 저장한다.
review_noun_tk = []
for n in review:
    review_noun_tk.append(okt.nouns(n)) # 명사 형태소만 출력

In [9]:
#한글자로만 추출된 데이터를 제거하기
review_noun_tk2 = []
for d in review_noun_tk:
    item = [i for i in d if len(i) > 1] # 뽑아낸 토큰 중 크기가 1 이상만 추출
    review_noun_tk2.append(item)

## LDA 모델 구축

In [10]:
# 명사 데이터를 사전화 하여 빈도 필터링을 진행한다.
dict = corpora.Dictionary(review_noun_tk2)

In [11]:
print(dict[1]) # 성공!

가끔


In [12]:
# 단어의 빈도수를 확인하기 위해 doc2bow 로 변환한다.
# BoW 모델은 문서의 단어의 빈도수를 표현해준다.
corpus = [dict.doc2bow(word) for word in review_noun_tk2]

In [13]:
# corpus 출력을 통해 (word_id, work_count)의 BoW 구성을 확인
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 4), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 2), (33, 2), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 3), (57, 1), (58, 1), (59, 2), (60, 2), (61, 3), (62, 1), (63, 2), (64, 2), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 4), (78, 2), (79, 1), (80, 1), (81, 1), (82, 4), (83, 1), (84, 1), (85, 3), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2), (93, 3), (94, 2), (95, 4), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 2), (102, 1), (103, 1), (104, 2), (105, 1), (106, 2), (107, 1), (108, 2), (109, 2), (110, 1)

In [17]:
# 토픽의 갯수 k 개 생성
k = 3

#gensim 라이브러리를 사용해 lda 모델을 생성하는 코드 각 옵션은 만들어놓은 
#corpus lda모델
#iterations 각 문서의 주제 할당을 반복하는 횟수
#num_topics 는 위에 지정한 k 값으로 토픽의 갯수
#id2word 각 토큰의 id에 해당하는 토큰을 연결해준다
#passes는 앞서 넣은 corpus를 학습 반복하는 횟수이다 수가 높아지면 더 잘 학습하지만 그만큼 시간이 증가한다.
#workers 쓰레드의 수 (작업자의 수)이다. 보통은 본인의 컴퓨터에 맞춰 설정하게 된다.
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, iterations = 12, num_topics = k, id2word = dict, passes = 5, workers = 10)

In [18]:
print(lda_model.print_topics(num_topics = k, num_words = 15))

[(0, '0.068*"등록" + 0.036*"헤드라인" + 0.030*"리뷰" + 0.025*"용이" + 0.023*"에어팟" + 0.017*"이어폰" + 0.014*"사용" + 0.013*"음질" + 0.013*"세대" + 0.012*"가격" + 0.011*"구매" + 0.011*"애플" + 0.009*"충전" + 0.009*"고민" + 0.008*"정품"'), (1, '0.056*"에어팟" + 0.037*"등록" + 0.032*"헤드라인" + 0.015*"구매" + 0.015*"프로" + 0.014*"배송" + 0.014*"사용" + 0.014*"이어폰" + 0.013*"세대" + 0.012*"음질" + 0.009*"충전" + 0.009*"제품" + 0.009*"정말" + 0.009*"소리" + 0.008*"선물"'), (2, '0.040*"에어팟" + 0.028*"세대" + 0.024*"사용" + 0.023*"구매" + 0.019*"프로" + 0.016*"음질" + 0.013*"제품" + 0.013*"이어폰" + 0.010*"배송" + 0.009*"애플" + 0.009*"고민" + 0.009*"아이폰" + 0.008*"가격" + 0.008*"쿠팡" + 0.007*"생각"')]


In [19]:
# lda모델을 디스플레이로 출력
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict)
pyLDAvis.display(lda_vis)